<a href="https://colab.research.google.com/github/blazingbhavneek/hyperbolic-ml/blob/main/L_Hydra_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version

Python 3.10.11


In [1]:
import zipfile

zip_file = '/content/L-hydra.zip'
target_dir = '/content/'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(target_dir)


In [3]:
# !python -m pip install snap-stanford
!pip install mpi4py
!mkdir /content/L-hydra/data/amazon/temp
%cd /content/L-hydra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365643 sha256=8b6158d8e895c3ce504d80551881d3d560c8ba23d0e9878b0348016db2b78232
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py
mkdir: cannot create directory ‘/content/L-hydra/data/amazon/temp’: File exists
/content/L-hydra


In [4]:
import numpy as np
import os
import sys, time, pdb, os, pickle
from numpy import *
import networkx as nx

###########################################################################
def randX(N,land, d):
  n = N-land
  n_temp = n
  param = 0.4  #distance thresold
  g = nx.generators.random_geometric_graph(n_temp, param, seed = 1)
  W = np.zeros((n_temp, n_temp))
  for (x,y) in g.edges:
    W[x][y] = np.random.uniform(0.1,3)
  W_hat = W + W.T
  L = np.diag(np.matmul(W_hat,np.ones(W_hat.shape[0]))) - W_hat
  print("L",L)
  np.random.seed(1)
  X_na = np.random.multivariate_normal(np.zeros(n), np.linalg.pinv(L),d+1)

  print("Size of original features", X_na.shape)

  np.random.seed(5)
  X_a = np.random.normal(0,1,size=(d+1,land))

  X = np.hstack((X_a, X_na))
  print("Xa shape",X_a.shape)

  for i in range(N):
    X[0,i] = np.sqrt(1+np.linalg.norm(X[1:d+1,i])**2);
  
  print("X shape",X.shape)
  print("X",X)
  return X

def x2hdm(N,land, X,d):

  n = N-land
  G = x2hgram(N,d,X)
  D = _arccosh(G)
  print("dcom shape", D)

  return D 

def _arccosh(G):
  D = np.arccosh(-G)
  return D

def x2hgram(N,d, X):
  n = N-d
  #######################################
  # print("shape of you", X.shape)
  X_ = X
  for n in range(N):
      x = X_[:,n]
      X_[:,n] = projectX(N,d,x)
  # print("shape of you after", X_.shape)
  #######################################
  G = x2lgram(N,d, X_)
  # print("valid G", G)
  #######################################
  E1 = G-valid(G) 
  # print('E1=',E1)
  if np.linalg.norm(E1,'fro') > 1e-10:
      print(np.linalg.norm(E1,'fro'))
      print('inaccuracy in  x2hgram - ')
  #######################################
  E2 = G-l_rankPrj(G)
  # print('E2=',E2)
  if np.linalg.norm(E2,'fro') > 1e-10:
      print(np.linalg.norm(E2,'fro'))
      print('inaccuracy in  x2hgram -- ')
  return G

def projectX(N,d,x):
  n= N-d
  
  #######################################
  H = np.eye(d+1)
  H[0,0] = -1
  I = np.eye(d+1)
  #######################################
  x0 = x[0]
  #print('x0 is:', x[0])
  eps = 1e-15
  center = 0
  error = abs(h_norm(x,N,d)+1)
  #print(error)
  A_opt = I
  for i in range(50):
      l = 10**(-i)
      if x0 > 0:
          lambda_min = max(center-l, -1+eps)
          lambda_max = min(center+l, 1-eps)
          number = 50
      else:
          lambda_min = max(center-l*10000, 1+eps)
          lambda_max = center+l*10000
          number = 100
      lambda_list = np.linspace(lambda_min,lambda_max,num=number)
      for lambda_ in lambda_list:
          A = np.linalg.inv(I + lambda_*H)
          x_l = np.matmul(A,x)
          if abs(h_norm(x_l,N,d)+1) < error:
              A_opt = A
              error = abs(h_norm(x_l,N,d)+1)
              center = lambda_
  x_opt = np.matmul(A_opt,x)
  if error > 1e-5:
      print('hi:',error)
      print(center)
  # print("shape of x_opt",x_opt.shape)    
  return x_opt

def x2lgram(N,d,X):
  n= N-d
  #######################################
  H = np.eye(d+1)
  H[0,0] = -1
  #######################################
  G = np.matmul(np.matmul(X.T,H),X)
  return G

def h_norm(x,N,d):
  n= N-d
  #######################################
  H = np.eye(d+1)
  H[0,0] = -1
  #######################################
  x_norm = np.matmul(np.matmul(x.T,H),x)
  return x_norm

def valid(G):
  np.fill_diagonal(G, -1)
  G[G >= -1] = -1
  
  return G

def l_rankPrj(G):
  X = lgram2x(N,d,G)
  
  return x2lgram(N,d,X)

def lgram2x(N,d,G):
  #######################################
  n= N-d
  #######################################
  w, v = np.linalg.eig(G)
  w = w.real
  v = v.real
  # print("d",d)
  # print("len w", len(w))
  lambda_0 = np.amin(w)
  ind_0 = np.argmin(w)
  w = np.delete(w, ind_0)
  ind = np.argsort(-w)
  w = -np.sort(-w)
  ind = ind[:d]
  w = w[:d]
  #######################################
  lambda_ = np.concatenate((abs(lambda_0), w), axis=None) 
  lambda_[lambda_ <= 0] = 0
  lambda_ = np.sqrt(lambda_)

  ind_ = np.concatenate((ind_0, ind+1), axis=None) 
  v = v[:, ind_]
  X = np.matmul(np.diag(lambda_),v.T)
  #######################################
  if X[0,0] < 0:
      X = -X
  #######################################
  return X                    
###########################################################################

In [5]:
# Parameters for Preprocessing
name = "amazon"
snapdir = "/content/L-hydra/data/amazon/snap/"
savedir = "/content/L-hydra/data/amazon/"
nprocs = 4
nlandmarks = 101
nNodes = 250
nvalidation = 40
maxprocs4seed = 8
split = True
nodes_per_run = 40      #Check splits and procs are similar
# Parameters for L-hydra / L-hydra+
curvature = 1
dim = 100
alpha = 1
equi_adj = 0.5
# Parameters for Parallelization of L-Hydra+
nrepeat = 8
graphsrc = 'amazon_edges' # original file from SNAP

In [6]:
with open('data/amazon/nNodes.txt', 'w') as f:
    f.write(str(nNodes))   

In [7]:
location = snapdir # directory for input data (contains edge list from snap)
dirloc = location
datafile = dirloc + graphsrc
newfilename = dirloc + graphsrc + '_remapped' # remapped edge list
filename4edges = dirloc + graphsrc
import numpy.random as random
metadata = {"nL":nlandmarks, "nVal":nvalidation, "datadir":savedir}
metadata["nnodes"] = nNodes
# Choose random seed for selecting landmarks and nonlandmark testing pairs
rn = random.RandomState(233)

filename_adjmat = name + '_edges_remapped'

In [8]:
N = nNodes
land = nlandmarks
n = N-land
d = dim-1
X = randX(nNodes,land, d)
print("X",X.shape)
D = x2hdm(nNodes,land, X,d)
print("D", D)

D = (D+D.T)/2
L2n = D[:nlandmarks,:]
print("L2n", L2n.shape)
print("L2n", L2n)

nNodes = nNodes
metadata["nnodes"] = nNodes

L [[ 61.1149311    0.           0.         ...  -2.86774057   0.
    0.        ]
 [  0.         101.7554889   -1.77854823 ...   0.          -2.06482899
    0.        ]
 [  0.          -1.77854823  99.96648083 ...  -2.17891179   0.
   -2.95650388]
 ...
 [ -2.86774057   0.          -2.17891179 ...  97.51578173   0.
    0.        ]
 [  0.          -2.06482899   0.         ...   0.          57.80682808
   -0.61368551]
 [  0.           0.          -2.95650388 ...   0.          -0.61368551
   95.01123299]]
Size of original features (100, 149)
Xa shape (100, 101)
X shape (100, 250)
X [[ 9.49370129e+00  9.01923884e+00  9.94081131e+00 ...  1.45132759e+00
   1.71011232e+00  1.42311736e+00]
 [ 1.70764452e-01 -1.75374086e+00  6.30296477e-01 ... -4.75269770e-02
  -1.12837353e-02  7.28999620e-03]
 [-6.10516401e-01 -1.70388541e+00  1.94437378e-01 ... -8.14066201e-02
   7.46976018e-02  7.10660972e-02]
 ...
 [-2.83795276e-01  2.45890322e-01 -4.24178925e-02 ...  3.15892278e-02
  -2.01152700e-01  7.93619

In [9]:
print("landmarks", nlandmarks)
print("total nodes",nNodes)
# print("distance matrix", )

landmarks 101
total nodes 250


In [10]:
import multiprocessing as mp
import sys, time, pdb, os, pickle

from numpy import *
from scipy.optimize import minimize, fmin_l_bfgs_b

'''
Functions used during embedding processes

First section: 
	functions used during pre and post processing
Second section: 
	objective functions for stress minimization
Third section: functions for landmark embedding
''' 

# **********************************
# Pre and post processing functions
# **********************************

def preprocdata(nodes, landmarks, nodes_per_run, dataloc, maxprocs4seed=1000, cleanup=False):
	print("\n", "*"*60)
	print("Pre-processing: splitting D mtx for n = %i and L = %i" %(nodes, landmarks))
	print("*"*60)

	if cleanup: clean(dataloc)
	
	# calculate the number of partitions for the D matrix
	n_split = int(ceil((nodes-landmarks)/float(nodes_per_run)))
	savetxt(dataloc + '/nonland_nsplit.txt', [n_split],fmt='%i')

	print("Splitting nonlandmark matrix")
	preproc(nodes, landmarks, dataloc, n_split, maxprocs4seed)

def landmark_preproc(nodes, landmarks, dim, dataloc):
	input_var = [" -n "," -L "," -d "," -s "]
	input_val = [nodes, landmarks, dim, dataloc]
	input_tot = ""
	for i,var in enumerate(input_var):
		input_tot += var + str(input_val[i])
	return input_tot

def nonlandmark_preproc(nodes, landmarks, dim, split, dataloc):
	input_var = [" -n "," -L "," -d "," -sp ", " -s "]
	input_val = [nodes, landmarks, dim, split, dataloc]
	input_tot = ""
	os.system('mkdir ' + os.path.join(dataloc, '/temp'))
	for i,var in enumerate(input_var):
		input_tot += var + str(input_val[i])
	return input_tot

def validation_preproc(nodes, landmarks, dim, nproc, dataloc, savedir):
	input_var = [" -n "," -L "," -d "," -p ", " -s1 ", " -s2 "]
	input_val = [nodes, landmarks, dim, nproc, dataloc, savedir]
	input_tot = ""
	for i,var in enumerate(input_var):
		input_tot += var + str(input_val[i])
	return input_tot

def save_obj(obj, name, savedir=''):
    with open(savedir + '/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name, savedir='' ):
    with open(savedir + '/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def clean(dataloc):
	print("Cleaning up data folder...")
	os.system('rm ' + os.path.join(dataloc, '*_split_*'))
	os.system('rm ' + os.path.join(dataloc, 'D_land.npy'))
	os.system('rm ' + os.path.join(dataloc, 'D_nonland.npy'))
	os.system('rm ' + os.path.join(dataloc, 'curv.txt'))
	os.system('rm ' + os.path.join(dataloc, 'd_nonland_true.txt'))
	os.system('rm ' + os.path.join(dataloc, '*rank*'))

def load_data_preproc(nodes, landmarks, dataloc):
	# Load full distance matrix for all nodes
	print("loading testing data...")
	D_test_points = loadtxt(os.path.join(dataloc, 'test_point_distances.txt'), dtype='int')
	D_test_pairs = loadtxt(os.path.join(dataloc, 'test_points.txt'), dtype='int')

	# define landmark ids and get distance matrix from only landmarks to nodes
	print("loading land_ids...")
	land_ids = loadtxt(os.path.join(dataloc, 'landmark_ids.txt'), dtype='int32')
	D_land2nodes = load(os.path.join(dataloc, 'dist_mtx_landmarks2nodes.npy'))

	print("finding nonland_ids...")
	ind = ~in1d(range(nodes),land_ids)
	print("length land_ids..." + str(len(land_ids)))
	print("length ind..." + str(len(ind)))
	print("length D_land2nodes..." + str(len(D_land2nodes)))
	D_nonland = D_land2nodes[:,ind]
	nonland_ids = arange(nodes)[ind]

	return D_land2nodes, D_land2nodes[:,land_ids], land_ids, D_nonland, nonland_ids, D_test_pairs, D_test_points

def load_data_main(nodes, landmarks, dataloc=''):
	# Load full distance matrix for all nodes
	D_test_points = loadtxt(os.path.join(dataloc, 'test_point_distances.txt'), dtype='int32')
	D_test_pairs = loadtxt(os.path.join(dataloc, 'test_points.txt'), dtype='int32')

	# define landmark ids and get distance matrix from only landmarks to nodes
	land_ids = loadtxt(os.path.join(dataloc, 'landmark_ids.txt'), dtype='int32')
	D_land = load(os.path.join(dataloc, 'D_land.npy')).astype('int')

	ind = ~in1d(range(nodes),land_ids)
	nonland_ids = arange(nodes)[ind]

	return D_land, land_ids, nonland_ids, D_test_pairs, D_test_points
 
def load_data_4land(nodes, landmarks, dataloc=''):
	# define landmark ids and get distance matrix from only landmarks to nodes
	land_ids = loadtxt(os.path.join(dataloc, 'landmark_ids.txt'), dtype='int32')
	D_land = load(os.path.join(dataloc, 'D_land.npy')).astype('int64')
	ind = ~in1d(range(nodes),land_ids)
	nonland_ids = arange(nodes)[ind]

	return D_land, land_ids, nonland_ids

def load_data_4nonland(nodes, landmarks, dataloc=''):
	# define landmark ids and get distance matrix from only landmarks to nodes
	land_ids = loadtxt(os.path.join(dataloc, 'landmark_ids.txt'), dtype='int32')
	ind = ~in1d(range(nodes),land_ids)
	nonland_ids = arange(nodes)[ind]

	return land_ids, nonland_ids

def split_nonland_mtx(nodes, landmarks, dataloc, n_split):
	node_ids = array_split(arange(nodes - landmarks),n_split)
	# load data and do pre-processing
	D_land2nodes, D_land, land_ids, D_nonland, nonland_ids, D_test_pairs0, D_test_dist0 = load_data_preproc(nodes, landmarks, dataloc)
	save(os.path.join(dataloc, 'D_nonland'), D_nonland)
	save(os.path.join(dataloc, 'D_land'), D_land)
	for ii,nn in enumerate(node_ids):
		print("Splitting %i out of %i" %(ii,len(node_ids)))
		D_nonland_temp = D_nonland[:,nn]
		save(os.path.join(dataloc, 'D_nonland_split_', str(ii).zfill(3)), D_nonland_temp)

	stemp = sum(D_nonland**2,0)
	print(stemp[:10])
	d_nonland_true = sqrt(abs(stemp)) # norm along rows, 
	savetxt(os.path.join(dataloc, 'd_nonland_true.txt'), d_nonland_true)

	return 0.0

def preproc(nodes, landmarks, dataloc, nprocs, maxprocs4seeds):
	node_ids = array_split(arange(nodes - landmarks),nprocs)
	# load data and do pre-processing
	print("Loading data...")
	D_land2nodes, D_land, land_ids, D_nonland, nonland_ids, D_test_pairs0, D_test_dist0 = load_data_preproc(nodes, landmarks, dataloc)
	print("Saving D_land and D_nonland...")
	print("Data types for nonlandmark and landmark matrices: ", D_nonland.dtype, D_land.dtype)
	save(os.path.join(dataloc, 'D_nonland'),D_nonland)
	save(os.path.join(dataloc, 'D_land'), D_land)
	for ii,nn in enumerate(node_ids):
		print("Splitting %i out of %i" %(ii+1,len(node_ids)))
		D_nonland_temp = D_nonland[:,nn]
		save(os.path.join(dataloc, 'D_nonland_split_' + str(ii).zfill(3)), D_nonland_temp)
	ns = 20
	ls = array_split(range(nodes-landmarks),ns)
	# square and sum chunks to avoid dtype errors (< 0 for int8)
	S = []
	for ii,lsi in enumerate(ls):
		print("squaring nonlandmark matrix %i out of %i" %(ii,ns))
		Dtemp = D_nonland[:,lsi].astype('int')
		S.append(sum(Dtemp**2,0))
	stemp = hstack(S)
	print(stemp[stemp < 0])
	d_nonland_true = sqrt(stemp) # norm along rows, 
	savetxt(os.path.join(dataloc, 'd_nonland_true.txt'), d_nonland_true)

	# generate random seed list
	print("generating random seed list...")
	seed_list = random.randint(1,max(maxprocs4seeds,1e4),maxprocs4seeds)
	savetxt(os.path.join(dataloc, 'random_seed_list.txt'), seed_list, fmt='%i')

	return 0.0

# **********************************
# Objective function definitions
# **********************************

def f(X,Y):
	return .5*linalg.norm(X-Y, ord='fro')**2

def df(X,Y):
	return X-Y

def hype_dist(p1,p2,curv):
	k = curv
	c = 1./sqrt(abs(k))
	x = dot(p1,p2)
	usq1 = dot(p1,p1) + 1.0
	usq2 = dot(p2,p2) + 1.0
	udotu = x - sqrt(usq1)*sqrt(usq2)
	acoshudotu = real(arccosh(-udotu + 0j))
	d = acoshudotu * c
	return d

def land_obj(x,**kwargs):
	dim = kwargs['dim']
	D0 = kwargs['D_landmark']
	grad = zeros(len(x))
	points = len(x[:-1])//dim
	k = x[-1]
	c = 1./sqrt(abs(k))
	P = x[:-1].reshape((points,dim))

	X = dot(P,P.T)
	Usq = diag(X) + 1.0
	U = sqrt(Usq)
	UdotU = X - outer(U,U)
	acoshUdotU = real(arccosh(-UdotU + 0j))
	fill_diagonal(acoshUdotU,0.0)

	D = acoshUdotU*c;
	D[isnan(D)] = 0
	dE = df(D,D0)
	err = f(D,D0)/2.0 # since matrix is symmetric

	# compute gradient w.r.t. k
	gradk = - dE * acoshUdotU * .5 * k * abs(k)**-2.5
	grad[-1] = sum(sum(gradk))/2.0 # divide by two since matrix is symmetric

	# compute gradient
	tol = 1e-12
	Atemp = UdotU**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./U,U)
	C = dE * A
	H1 = C * B
	hsum = sum(H1,1)
	L = len(D0)
	Ci = zeros((len(C),len(C)))
	for i in range(L):
		ui = P[i,:]
		hi = hsum[i]
		fill_diagonal(Ci,C[i])
		g = sum(dot(P.T,Ci),1) - hi*ui
		grad[dim*i:(i+1)*dim] = g

	return err, grad

def nonland_obj(x, node, **kwargs):
	'''
	computes nonlandmark error for an entire nonlandmark
	matrix D0
	'''
	# node is index of D_nonland not D_land2nodes
	dim = kwargs['dim']
	D0 = kwargs['D_nonland']
	L = kwargs['Land_points']
	n_land, dim = L.shape
	nl_id = node
	k = kwargs['k']
	c = 1./sqrt(abs(k))

	X = dot(L,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(L.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	d0 = D0[:,nl_id]
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(L.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

def nonland_obj_percol(x, dim, L, k, d0):
	'''
	computes nonlandmark error for fixed column d0 of the 
	nonlandmark matrix
	'''
	land_idx = arange(len(L))
	Lnew = L[land_idx,:]
	n_land, dim = Lnew.shape
	c = 1./sqrt(abs(k))

	X = dot(Lnew,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(Lnew.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(Lnew.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

def pre_nonland_obj_percol(x, dim, L, k, rL_in_land_idx, d0):
	'''
	computes nonlandmark error for fixed column d0 of the 
	nonlandmark matrix
	'''
	Lnew = L[rL_in_land_idx,:]
	d0 = d0[rL_in_land_idx]
	n_land, dim = Lnew.shape
	c = 1./sqrt(abs(k))

	X = dot(Lnew,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(Lnew.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(Lnew.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

# **********************************
# Landmark Embedding functions
# **********************************

def embed_prelandmarks(xstart,factr,param_list):
	def f(x):
		return land_obj(x,**param_list)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(f,xstart, bounds=bnds, factr=factr)
	return x,f,d
	
def embed_pre_nonlandmarks(xstart,node,param_list2):
	def g(x):
		return nonland_obj(x,node,**param_list2)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(g,xstart, bounds=bnds, factr=1e12)
	return x,f,d

def pre_embedding(dim, D_land, landmarks0, xstart, seed=133):
	D0 = D_land
	x0 = xstart
	nL = len(D0)
	rn = random.RandomState(seed)
	if nL <= 32:
		x = append(rn.rand(1), x0.flatten())
		param_list = {'dim': dim, 'D_landmark' : D_land}
		x1,_,_ = embed_prelandmarks(x,1e10,param_list)
		return x1
	else:
		# find nearest power of two and embed
		exp = floor(log2(nL))
		if abs(exp - log2(nL)) == 0:
			# if nL is a perfect power of 2, then use lowest exp
			rL = int(2**(exp-1))
		else:
			rL = int(2**(exp))
		land_ids_new = array(sort(rn.permutation(nL)[:rL])).astype('int')
		D_land2nodes_new = D0[land_ids_new,:]
		D_land_new = D_land2nodes_new[:,land_ids_new]
		xstart_new = x0[land_ids_new]
		# embed new set of sub-landmarks
		param_list_new = {'dim': dim, 'D_landmark' : D_land_new, 'x_start0' : xstart_new}
		x_land_soln = pre_embedding(dim, D_land_new,landmarks0,xstart_new)
		# solve exactly (comment out for faster runs)
		if rL > 32 and rL <= int(landmarks0/2):
			x1temp,_,_ = embed_prelandmarks(x_land_soln,1e10,param_list_new)
			x_land_soln = x1temp
		L_new = x_land_soln[:-1].reshape((rL,dim))
		curv_new = x_land_soln[-1]
		# embed nonlandmarks
		nonland_ids_new = delete(arange(nL),land_ids_new)
		D_nonland_new = D_land2nodes_new[:,nonland_ids_new]
		nl_ids = range(len(nonland_ids_new))
		nonland_xmin = []
		for nl_id in nl_ids:
			param_list2_new = {'dim': dim, 'Land_points': L_new, 'D_nonland': D_nonland_new, 'k': curv_new}
			xstart2_0 = x0[nonland_ids_new[nl_id]]
			x2,_,_ = embed_pre_nonlandmarks(xstart2_0,nl_id,param_list2_new)
			nonland_xmin.append(x2)
		# fill in embedded points
		C0 = zeros((nL,dim))
		for ii,id in enumerate(land_ids_new):
			C0[id,:] = L_new[ii,:]
		for ii,xmin_nl in enumerate(nonland_xmin):
			C0[nonland_ids_new[ii],:] = xmin_nl
		return append(C0.flatten(),curv_new)

def pre_embed(s, param_list):
	''' 
	Call pre_embedding 
	'''
	dim = param_list['dim']
	D_land = param_list['D_landmark']
	landmarks0 = param_list['landmarks0']
	xstart = param_list['x_start0']
	x = pre_embedding(dim,D_land,landmarks0,xstart,seed=s)
	return land_obj(x,**param_list)[0], x

def embed_landmarks(xstart,param_list,factr=1e10):
	def f(x):
		return land_obj(x,**param_list)
	bnds = [ (None, None) for i in range(len(xstart))]
	x,f,d = fmin_l_bfgs_b(f,xstart,\
							bounds=bnds,\
							factr=factr)
	return f,x

def F(args):
	'''
	Main embedding algorithm (run for multiple seeds)
	'''
	seed = args[0]
	param_list = args[1]
	f0,x0 = pre_embed(seed, param_list)
	return embed_landmarks(x0,param_list)

def run_landmark_embedding(param_list, np, nodes, land_ids, dataloc):
	start = time.time()
	rn_test = random.RandomState(163)
	inputs1 = [[rn_test.randint(1e3),param_list] for i in range(np)]
	pool = mp.Pool(processes=np)
	solns = pool.map(F, inputs1)
	fvals = [output[0] for output in solns]
	min_index = argmin(fvals)
	fval,xsoln = solns[min_index]
	land_error = fval
	dim = param_list['dim']
	D_land = param_list['D_landmark']
	landmarks = len(D_land)
	Xsoln = xsoln[:-1].reshape(landmarks,dim)
	csoln = xsoln[-1]
	landmark_rel_error = sqrt(2*land_error)/linalg.norm(D_land,ord='fro')
	print("Total landmark time is %3.5f seconds" %(time.time()-start))
	C = zeros((nodes, dim))
	C[land_ids,:] = Xsoln
	savetxt(os.path.join(dataloc, 'curv.txt'),[csoln])
	savetxt(os.path.join(dataloc, 'fval.txt'),[fval])
	save(os.path.join(dataloc, 'C'),C)
	save(os.path.join(dataloc, 'L_points'),C[land_ids,:])

	hypy_land_time = time.time() - start
	savetxt(os.path.join(dataloc, 'hypy_land_time.txt'),[hypy_land_time])
	savetxt(os.path.join(dataloc, 'landmark_rel_error.txt'),[landmark_rel_error])

	L_points = C[land_ids]
	curv = csoln
	pool.close()

	return hypy_land_time, landmark_rel_error, L_points, curv, C

def run_single_landmark_embedding(param_list, nodes, land_ids, dataloc, seed):
	start = time.time()

	fval, xsoln = F((seed,param_list))
	land_error = fval
	dim = param_list['dim']
	D_land = param_list['D_landmark']
	landmarks = len(D_land)
	Xsoln = xsoln[:-1].reshape(landmarks,dim)
	csoln = xsoln[-1]
	landmark_rel_error = sqrt(2*land_error)/linalg.norm(D_land,ord='fro')
	hypy_land_time = time.time() - start

	return hypy_land_time, landmark_rel_error, Xsoln, csoln

def gather_landmarks(nodes, landmarks, dim, dataloc, nprocs):
	'''
	Get optimal embedding out of different ranks
	'''
	fvals = zeros(nprocs)
	land_times = zeros(nprocs)
	for r in range(nprocs):
		fvals[r] = loadtxt(os.path.join(dataloc, 'fval_rank' + str(r) + '.txt'))
		land_times[r] = loadtxt(os.path.join(dataloc, 'land_time_rank' + str(r) + '.txt'))
	opt_r = argmin(fvals) # optimal (min) error
	fopt = fvals[opt_r] # relative error (not actual objective fun)
	topt = amax(land_times)
	copt = loadtxt(os.path.join(dataloc, 'curv_rank' + str(opt_r) + '.txt')) # optimal curvature
	land_coord = load(os.path.join(dataloc, 'land_coord_rank' + str(opt_r) + '.npy'))
	savetxt(os.path.join(dataloc, 'landmark_rel_error.txt'), [fopt])

	# save optimal landmark coordinates and curvature
	save(os.path.join(dataloc, 'land_coord_opt'), land_coord)
	savetxt(os.path.join(dataloc, 'copt.txt'), [copt])
	savetxt(os.path.join(dataloc, 'hypy_curv.txt'), [copt])
	savetxt(os.path.join(dataloc, 'fopt.txt'), [fopt])
	savetxt(os.path.join(dataloc, 'hypy_land_time.txt'), [amax(land_times)])
	return fopt, land_coord, copt, topt, opt_r

def gather_nonlandmarks_split(nodes, landmarks, dim, dataloc, split, nprocs):
	F2vals = []
	X2solns = []
	NLtimes = []
	for r in range(nprocs):
		ftemp = load(os.path.join(dataloc, 'temp', 'f2s_split' + str(split) + '_rank' + str(r) + '.npy'))
		xtemp = load(os.path.join(dataloc, 'temp', 'x2s_split' + str(split) + '_rank' + str(r) + '.npy'))
		timetemp = loadtxt(os.path.join(dataloc, 'temp', 'nonland_time_split' + str(split) + '_rank' + str(r) + '.txt'))
		F2vals.append(ftemp)
		X2solns.append(xtemp)
		NLtimes.append(float(timetemp))
	f2s = hstack(F2vals).flatten()
	x2s = vstack(X2solns)
	hypy_nonland_time = amax(NLtimes)

	save(os.path.join(dataloc, 'f2vals_split_' + str(split)), f2s)
	save(os.path.join(dataloc, 'x2solns_split_' + str(split)), x2s)
	save(os.path.join(dataloc, 'nonland_time_split_' + str(split)), hypy_nonland_time)

	return f2s, x2s, hypy_nonland_time

def gather_nonlandmarks(nodes, landmarks, dim, dataloc, n_split):
	F2vals = []
	X2solns = []
	NLtimes = []
	for nn in range(n_split):
		ftemp = load(os.path.join(dataloc, 'f2vals_split_' + str(nn) + '.npy'))
		xtemp = load(os.path.join(dataloc, 'x2solns_split_' + str(nn) + '.npy'))
		timetemp = load(os.path.join(dataloc, 'nonland_time_split_' + str(nn) + '.npy'))
		F2vals.append(ftemp)
		X2solns.append(xtemp)
		NLtimes.append(float(timetemp))

	f2s = hstack(F2vals).flatten()
	x2s = vstack(X2solns)
	hypy_nonland_time = sum(NLtimes)

	d_nonland_true = loadtxt(os.path.join(dataloc, 'd_nonland_true.txt'))
	nonland_avg_rel_error = mean(sqrt(2*f2s).flatten()/d_nonland_true)

	print("Total nonlandmark time is %3.5f seconds" %hypy_nonland_time)
	
	# add nonland coordinates to main embedding
	land_ids = loadtxt(os.path.join(dataloc, 'landmark_ids.txt')).astype('int')
	nonland_ids = loadtxt(os.path.join(dataloc, 'nonlandmark_ids.txt')).astype('int')
	land_coord = load(os.path.join(dataloc, 'land_coord_opt.npy'))
	C = zeros((nodes, dim))
	C[land_ids,:] = land_coord
	C[nonland_ids,:] = x2s
	save(os.path.join(dataloc, 'hypy_coord'), C)
	savetxt(os.path.join(dataloc, 'hypy_nonland_time.txt'), [hypy_nonland_time])
	savetxt(os.path.join(dataloc, 'nonland_avg_rel_error.txt'), [nonland_avg_rel_error])

	print("Non-landmark error: ", nonland_avg_rel_error)
	return C, nonland_avg_rel_error, hypy_nonland_time

def cleanup_at_end(dataloc):
	print("Cleaning up data folder...")
	os.system('rm ' + os.path.join(dataloc, 'temp/*.*'))
	os.system('rm ' + os.path.join(dataloc, '*opt*'))
	os.system('rm ' + os.path.join(dataloc, 'd_nonland_true.txt'))
	os.system('rm ' + os.path.join(dataloc, '*split*'))
	os.system('rm ' + os.path.join(dataloc, '*rank*'))

In [11]:
import numpy as np 
import numpy.random as random
import matplotlib.pyplot as mpl
import multiprocessing as mp
import pandas as pd
import os, pdb, time, pickle, sys, argparse

class Preprocessing:

	def __init__(self, name, indir, outdir, nprocs, nL, nVal):
		print("Start Preprocessing")
		self.name = name # network name
		self.location = indir # directory for input data (contains edge list from snap)
		self.datadir = outdir # directory for output data
		self.nprocs = nprocs # number of processes for parallel embedding
		self.nL = nL # number of landmarks
		self.nVal = nVal # number of validation pairs
	
	def ReMapNodes(self, filename):

		graphsrc = filename # original file from SNAP
		nprocs = self.nprocs # processes
		dirloc = self.location
		datafile = dirloc + graphsrc
		newfilename = dirloc + graphsrc + '_remapped' # remapped edge list

		edge_list, nodes, n_edges = self.load_graph_data(datafile)

		# create dictionary map from old nodes to new nodes
		global D
		D = {}
		for i,n in enumerate(nodes):
			D[n] = i

		# remap function requires dictionary D as a global
		start = time.time()
		new_edge_list = self.remap_edge_list(edge_list, nprocs)
		end = time.time()
		print("Total time for remapping is %f seconds" %(end-start))

		# save data
		print('writing to text file...')
		np.savetxt(newfilename,new_edge_list,fmt='%i',delimiter='\t')

		self.edge_list_file = filename + '_remapped'
		self.edge_list_location = self.location
		
	def GenAdjMat(self, filename):
		'''
		Compute the landmarks given the edge list along
		with distances from nodes to each landmark
		'''
		self.metadata = {"nL":self.nL, "nVal":self.nVal, "datadir":self.datadir}

		os.system('python lm_emb_preprocessing_gen_adj.py' + ' -filename=' + filename  \
											+ ' -location=' + self.location  \
											+ ' -datadir=' + self.datadir   \
											+ ' -nprocs=' + str(self.nprocs) \
											+ ' -nL=' + str(self.nL) \
											+ ' -nval=' + str(self.nVal))
											
		self.nnodes = np.int(np.loadtxt(self.datadir + 'nNodes.txt'))
		self.metadata["nnodes"] = self.nnodes
		self.save_meta(self.metadata, "metadata")
		
	def set_main_env(self, maxprocs4seed, split, nodes_per_run):
		'''
		Split the nonlandmark embedding into serial chunks

		If split = True, then user has to define nodes_per_run. Split
		defines how many serial chunks to perform the nonlandmark
		embedding. Each chunk is then performed in parallel. E.g., 
		if nnodes = 100 and nodes per run is 2, then the first 50
		nodes are embedding in parallel, followed by the next 50.

		Most data is written to the datadir so that there is minimal
		sharing of data and each function can be run at different times.  
		'''
		# set hyperbolic dimension and processors for nonland embedding
		nodes = self.metadata['nnodes']
		landmarks = self.metadata['nL']
		datadir = self.metadata['datadir']
		self._nprocs1 = maxprocs4seed # parallel landmark procs

		if split==True:
			self.nl_split = int(np.ceil((nodes-landmarks)/float(nodes_per_run)))
			# preprocess data for landmark and nonlandmark embedding
			os.system('mkdir ' + os.path.join(datadir, 'temp'))
			lm_emb_func.preprocdata(nodes, landmarks, nodes_per_run, datadir, maxprocs4seed=100*self._nprocs1, cleanup=True)
		
	def save_meta(self, obj, name):
	    with open(os.path.join(self.datadir, name + ".pkl"), 'wb') as f:
	    	pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

	def load_meta(self, name):
	    with open(os.path.join(self.datadir, name + ".pkl"), 'rb') as f:
	    	return pickle.load(f)
	
	def split(self, seq, procs):
		'''
		for parallel proccessing of re-labeling
		seq is a list of input parameters
		procs are the number of processors
		'''
		avg = len(seq) / float(procs)
		out = []
		last = 0.0
		while last < len(seq):
			out.append(seq[int(last):int(last + avg)])
			last += avg
		return out
    	
	def load_graph_data(self, filename):
		print('Loading data...')
		e0 = pd.read_csv(filename,sep='\t',header=None,skiprows=4,dtype='int32')
		e1 = np.array(e0[0],dtype='int32')   
		e1.shape = (len(e1),1)
		e2 = np.array(e0[1],dtype='int32')
		e2.shape = (len(e2),1)   
		edge_list = np.hstack((e1,e2)).astype('int32')
		n_edges = len(edge_list)
		nodes = np.unique(edge_list.flatten()) # get number of nodes
		print('Done loading data!')

		return edge_list, nodes, n_edges

	def get_new_edge_list(self, edge_list_temp):
		''' 
		renumber edge list using dictionary of indices
		'''
		new_edge_list = np.zeros(edge_list_temp.shape,dtype='int32')
		for j,r in enumerate(edge_list_temp):
			if j % 100000 == 0: print("%.2f percent done" %(float(j)/len(edge_list_temp)))
			new_edge_list[j,0] = D[r[0]]
			new_edge_list[j,1] = D[r[1]]
		return new_edge_list

	def remap_edge_list(self, edge_list, nprocs=4):
		''' 
		re-map edge list to have consecutive numbers (in parallel)
		create a dictionary which maps node # to index 
		note that the node number might be higher than the total number of 
		unique n_nodes, which is why we are doing this
		'''
		sp = self.split(range(len(edge_list)),nprocs)
		edge_split = [edge_list[s,:] for s in sp]
		pool = mp.Pool(processes=nprocs)
		new_edge_lists = pool.map(self.get_new_edge_list,edge_split)
		new_edge_list = np.vstack(new_edge_lists)

		return new_edge_list

In [12]:
preproc = Preprocessing(name, snapdir, savedir, nprocs, nlandmarks, nvalidation)
preproc.metadata = metadata

Start Preprocessing


In [13]:
def clean(dataloc):
    print("Cleaning up data folder...")
    os.system('rm ' + os.path.join(dataloc, '*_split_*'))
    os.system('rm ' + os.path.join(dataloc, 'D_land.npy'))
    os.system('rm ' + os.path.join(dataloc, 'D_nonland.npy'))
    os.system('rm ' + os.path.join(dataloc, 'curv.txt'))
    os.system('rm ' + os.path.join(dataloc, 'd_nonland_true.txt'))
    os.system('rm ' + os.path.join(dataloc, '*rank*'))
 
def preprocdata(nodes, landmarks, nodes_per_run, dataloc, maxprocs4seed=1000, cleanup=False):
    print("\n", "*"*60)
    print("Pre-processing: splitting D mtx for n = %i and L = %i" %(nodes, landmarks))
    print("*"*60)

    if cleanup: clean(dataloc)
    
    # calculate the number of partitions for the D matrix
    n_split = int(ceil((nodes-landmarks)/float(nodes_per_run)))
    savetxt(dataloc + '/nonland_nsplit.txt', [n_split],fmt='%i')
    print("Splitting nonlandmark matrix")
    preproc(nodes, landmarks, dataloc, n_split, maxprocs4seed)

def preproc(nodes, landmarks, dataloc, nprocs, maxprocs4seeds):
    node_ids = array_split(arange(nodes - landmarks),nprocs)
    # load data and do pre-processing
    print("Loading data...")
    land_ids = [i for i in range(nlandmarks)]
    non_land_ids = [i for i in range(nlandmarks,nNodes)]
    D_land2nodes, D_land, land_ids, D_nonland, nonland_ids = L2n,L2n[:,:nlandmarks],land_ids,L2n[:,nlandmarks:],non_land_ids 
    print("Shape of D_land",D_land.shape)
    print("Shape of D_nonland",D_nonland.shape)
    print("Saving D_land and D_nonland...")
    print("Data types for nonlandmark and landmark matrices: ", D_nonland.dtype, D_land.dtype)
    save(os.path.join(dataloc, 'D_nonland'),D_nonland)
    save(os.path.join(dataloc, 'D_land'), D_land)
    for ii,nn in enumerate(node_ids):
        print("Splitting %i out of %i" %(ii+1,len(node_ids)))
        D_nonland_temp = D_nonland[:,nn]
        save(os.path.join(dataloc, 'D_nonland_split_' + str(ii).zfill(3)), D_nonland_temp)
    ns = 20
    ls = array_split(range(nodes-landmarks),ns)
    # square and sum chunks to avoid dtype errors (< 0 for int8)
    S = []
    for ii,lsi in enumerate(ls):
        print("squaring nonlandmark matrix %i out of %i" %(ii,ns))
        Dtemp = D_nonland[:,lsi].astype('int')
        S.append(sum(Dtemp**2,0))
    stemp = hstack(S)
    print(stemp[stemp < 0])
    d_nonland_true = sqrt(stemp) # norm along rows, 
    savetxt(os.path.join(dataloc, 'd_nonland_true.txt'), d_nonland_true)

    # generate random seed list
    print("generating random seed list...")
    seed_list = random.randint(1,max(maxprocs4seeds,1e4),maxprocs4seeds)
    savetxt(os.path.join(dataloc, 'random_seed_list.txt'), seed_list, fmt='%i')

    return 0.0

In [14]:
datadir = metadata['datadir']
nprocs1 = maxprocs4seed
preprocdata(nNodes, nlandmarks, nodes_per_run, datadir, maxprocs4seed=100*nprocs1, cleanup=True)


 ************************************************************
Pre-processing: splitting D mtx for n = 250 and L = 101
************************************************************
Cleaning up data folder...
Splitting nonlandmark matrix
Loading data...
Shape of D_land (101, 101)
Shape of D_nonland (101, 149)
Saving D_land and D_nonland...
Data types for nonlandmark and landmark matrices:  float64 float64
Splitting 1 out of 4
Splitting 2 out of 4
Splitting 3 out of 4
Splitting 4 out of 4
squaring nonlandmark matrix 0 out of 20
squaring nonlandmark matrix 1 out of 20
squaring nonlandmark matrix 2 out of 20
squaring nonlandmark matrix 3 out of 20
squaring nonlandmark matrix 4 out of 20
squaring nonlandmark matrix 5 out of 20
squaring nonlandmark matrix 6 out of 20
squaring nonlandmark matrix 7 out of 20
squaring nonlandmark matrix 8 out of 20
squaring nonlandmark matrix 9 out of 20
squaring nonlandmark matrix 10 out of 20
squaring nonlandmark matrix 11 out of 20
squaring nonlandmark matrix

In [15]:
import numpy as np 
import os, math, warnings

from scipy.linalg import eigh
from scipy.optimize import minimize
class LHydra:

  def __init__(self, name, indir, outdir, nprocs):
    print("Start Hydra-Embedding")
    self.name = name
    self.snapdir = indir
    self.datadir = outdir
    self.nprocs = nprocs
    
  def hydra_landmark(self, dim=2, curvature=1.0, alpha=1.1, equi_adj=0.5, polar=False, isotropic_adj=True, hydra=False, lorentz=False):
    ''' 
    Wrapper function for different variants of the L-hydra method.
    
    If the curvature is 'None', L-hydra tries to find the optimal curvature.
    Otherwise, L-hydra is run with fixed curvature.
    '''
    if curvature != None:
      return self.hydra_landmark_fixed_curvature(curvature, dim, alpha, equi_adj, polar, isotropic_adj, hydra, lorentz)
    else:
      # setup control parameter for curvature optimization
      hydra=False
      lorentz = False
      
      # initial value for curvature 
      curvature = 1
      
      eps = np.finfo(np.double).eps
      D_land = np.load(os.path.join(self.datadir, 'D_land.npy')).astype('int')
      k_bounds = [eps,max(1,(8/D_land.max())**2)] # a priori bounds for curvature
      k_min = minimize(
        fun=self.hydra_landmark_fixed_curvature,
        args=(dim, alpha, equi_adj, polar, isotropic_adj, hydra, lorentz),
        x0=curvature,
        method="BFGS",
        bounds=k_bounds,
        options={"disp": True, "maxiter": 1000}
      )
      k1_objective = self.hydra_landmark_fixed_curvature(1, dim, alpha, equi_adj, polar, isotropic_adj, hydra, lorentz)
      k_optimal = k_min.x[0]
      if k1_objective < k_min.fun: 
        k_optimal = 1 # make sure that returned result is never worse than unit curvature
      return self.hydra_landmark_fixed_curvature(k_optimal, dim, alpha, equi_adj, polar, isotropic_adj, hydra, lorentz)
        
  def hydra_landmark_fixed_curvature(self, curvature=1.0, dim=2, alpha=1.1, equi_adj=0.5, polar=False, isotropic_adj=True, hydra=False, lorentz=False):
    
    # load distance matrices and landmark indices (as computed during pre-processing)
    D_land = np.load(os.path.join(self.datadir, 'D_land.npy')).astype('int') # distances between landmark nodes
    D_nonland = np.load(os.path.join(self.datadir, 'D_nonland.npy')).astype('int') # distances between landmark and non-landmark nodes
    print("shape of dland", D_land.shape)
    print("shape of dnonland", D_nonland.shape)

    nodes = np.loadtxt(os.path.join(self.datadir, 'nNodes.txt'),dtype='int32') # number of network nodes
    land_ids = np.loadtxt(os.path.join(self.datadir, 'landmark_ids.txt'),dtype='int32') # indices of landmark nodes
    
    ind = ~np.in1d(range(nodes),land_ids)
    nonland_ids = np.arange(nodes)[ind] # indices of non-landmark nodes

    # sanitize/check input
    if any(np.diag(D_land) != 0):  # non-zero diagonal elements are set to zero
      np.fill_diagonal(D_land, 0)
      warnings.warn("Diagonal of input matrix D_land has been set to zero")
      
    if dim > len(D_land):
      raise RuntimeError(
        f"Hydra cannot embed {len(D_land)} points in {dim}-dimensions. Limit of {len(D_land)}."
      )
    
    if not np.allclose(D_land, np.transpose(D_land)):
      warnings.warn(
        "Input matrix D_land is not symmetric.\
        Lower triangle part is used."
      )

    if dim > 2:
      # set default values in dimension > 2
      isotropic_adj = False
      if polar:
        warnings.warn("Polar coordinates only valid in dimension two")
        polar = False
      if equi_adj != 0.0:
        warnings.warn("Equiangular adjustment only possible in dimension two.")
      
    # convert distance matrix to 'hyperbolic Gram matrix'
    A_land = np.cosh(np.sqrt(abs(curvature))*D_land)
    print("shape of A_land", A_land.shape)
    A_nonland = np.cosh(np.sqrt(abs(curvature))*D_nonland)
    print("shape of A_nonland", A_nonland.shape)
    nlm = len(land_ids)
    nnonlm = nodes - nlm
    
    # check for large/infinite values
    A_max = np.amax(A_land)
    if A_max > 1e8:
      warnings.warn(
        "Gram Matrix contains values > 1e8. Rerun with smaller\
        curvature parameter or rescaled distances."
      )
    if A_max == float("inf"):
      warnings.warn(
        "Gram matrix contains infinite values.\
        Rerun with smaller curvature parameter or rescaled distances."
      )

    # Eigendecomposition of A
    # compute leading Eigenvalue and Eigenvector
    lambda0, x0 = eigh(A_land, subset_by_index=[nlm-1, nlm-1])
    # compute lower tail of spectrum
    w, v = eigh(A_land, subset_by_index=[0,dim-1])
    print("w",len(w))
    print("v", len(v))
    idx = w.argsort()[::-1]
    # print("idx",len(idx))
    spec_tail = w[idx] # Last dim Eigenvalues
    print("")
    X_land_raw = v[:,idx] # Last dim Eigenvectors
    print("shape of  X_land_raw",  X_land_raw.shape) 
    
    x0 = x0 * np.sqrt(lambda0) # scale by Eigenvalue
    if x0[0]<0:
      x0 = -x0 # Flip sign if first element negative

    # no isotropic adjustment: rescale Eigenvectors by Eigenvalues
    if not isotropic_adj:
      if np.array([spec_tail > 0]).any():
        warnings.warn(
          "Spectral Values have been truncated to zero. Try to use\
          lower embedding dimension"
        )
        spec_tail[spec_tail > 0] = 0
      X_land_raw = np.matmul(X_land_raw, np.diag(np.sqrt(np.maximum(-spec_tail,0))))
      
    X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])
    
    X_raw = np.zeros((nodes, dim))

    print(len(land_ids))
    X_raw[land_ids,:] = X_land_raw
    
    X_raw[nonland_ids,:] = X_nonland[:,1:(dim+1)]
    
    x0_full = np.zeros((nodes, 1))
    x0_full[land_ids,0] = x0[:,0]
    x0_full[nonland_ids,0] = X_nonland[:,0]
    x_min = x0_full.min()
    
    
    if hydra:
      # Calculate radial coordinate
      s = np.sqrt(np.sum(X_raw ** 2, axis=1))
      directional = X_raw / s[:, None]  # convert to directional coordinates
      r = np.sqrt((alpha*x0_full - x_min)/(alpha*x0_full + x_min)) ## multiplicative adjustment (scaling)
      X = self.poincare_to_hyper(r=r, directional=directional)
    else:
      X = X_raw
      
    # Calculate polar coordinates if dimension is 2
    if dim == 2:
      # calculate polar angle
      theta = np.arctan2(X[:, 0], -X[:, 1])

      # Equiangular adjustment
      if equi_adj > 0.0:
        angles = [(2 * x / nodes - 1) * math.pi for x in range(0, nodes)]
        theta_equi = np.array(
          [x for _, x in sorted(zip(theta, angles))]
        )  # Equi-spaced angles
        # convex combination of original and equi-spaced angles
        theta = (1 - equi_adj) * theta + equi_adj * theta_equi
        # update directional coordinate
        directional = np.array([np.cos(theta), np.sin(theta)]).transpose()
        
    if lorentz:
      X_lorentz = np.concatenate((x0_full, X), axis=1)
    
    np.save(os.path.join(self.datadir, 'xstart'), X)
    print('*** DIMENSIONS OF XSTART ' + str(X.shape[0]) + ' and ' + str(X.shape[1]))
    self.split_xstart(nodes, nlm, self.datadir, self.nprocs)
    
    #stress = self.get_stress(curvature, X_raw[land_ids,:], D_land)
    
    return X
    
  def split_xstart(self, nodes, landmarks, dataloc, nprocs):
    '''
    Splits (hyperbolic) coordinates into nprocs chunks of equal size
    '''
    node_ids = np.array_split(np.arange(nodes - landmarks),nprocs)
    print("Loading hydra results...")
    xstart_full = np.load(os.path.join(dataloc, 'xstart.npy'))
    nonland_ids = np.loadtxt(os.path.join(dataloc, 'nonlandmark_ids.txt'), dtype='int32')
    xstart = xstart_full[nonland_ids,:]
    print("Length of starting point:" + str(len(xstart)))
    for ii,nn in enumerate(node_ids):
      print("Splitting %i out of %i" %(ii+1,len(node_ids)))
      xstart_tmp = xstart[nn,:]
      np.save(os.path.join(dataloc, 'xstart_split_' + str(ii).zfill(3)), xstart_tmp)

  def get_stress(self, curvature, x, dist):
    '''
    Calculate stress of embedding from given coordinates
    '''
    X = np.matmul(x, x.T)
    u_tilde = np.sqrt(X.diagonal() + 1)
    H = X - np.outer(u_tilde, u_tilde)
    D = 1 / np.sqrt(abs(curvature)) * np.arccosh(np.maximum(-H, 1))
    np.fill_diagonal(D, 0)
    y = 0.5 * np.sum((D - dist) ** 2)
    return y
    
  def poincare_to_hyper(self, r, directional):
    '''
    Convert coordinates in the Poincare ball to reduced hyperbolic coordinates
    '''
    X = (directional.transpose()*(2*r/(1-r**2))).transpose()
    return X

In [16]:
# from lm_emb_hydra import LHydra

hydra = LHydra(name, snapdir, savedir, nprocs)
hydra.hydra_landmark(dim, curvature, alpha, equi_adj)

Start Hydra-Embedding
shape of dland (101, 101)
shape of dnonland (101, 149)
shape of A_land (101, 101)
shape of A_nonland (101, 149)
w 100
v 101

shape of  X_land_raw (101, 100)
101
*** DIMENSIONS OF XSTART 250 and 100
Loading hydra results...
Length of starting point:149
Splitting 1 out of 4
Splitting 2 out of 4
Splitting 3 out of 4
Splitting 4 out of 4


<ipython-input-15-354cfca85f42>:86: UserWarning: Equiangular adjustment only possible in dimension two.
  warnings.warn("Equiangular adjustment only possible in dimension two.")
<ipython-input-15-354cfca85f42>:130: UserWarning: Spectral Values have been truncated to zero. Try to use          lower embedding dimension
  warnings.warn(
<ipython-input-15-354cfca85f42>:137: RuntimeWarning: invalid value encountered in true_divide
  X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])


array([[ 0.        ,  0.        ,  0.        , ...,  0.74775388,
         2.56038753,  0.86584783],
       [ 0.        ,  0.        ,  0.        , ...,  0.78096437,
         3.28448747, -3.60999046],
       [ 0.        ,  0.        ,  0.        , ..., -1.93837997,
         2.01879049,  0.70188553],
       ...,
       [        nan,         nan,         nan, ..., -0.06641681,
        -0.05190234,  0.09040573],
       [        nan,         nan,         nan, ..., -0.06641681,
        -0.05190234,  0.09040573],
       [        nan,         nan,         nan, ..., -0.06641681,
        -0.05190234,  0.09040573]])

In [17]:
from lm_emb_hydra_plus import LHydraPlus
# Run Hydra Plus
hydra_plus = LHydraPlus(name, savedir, nlandmarks, nodes_per_run)

Start HydraPlus-Embedding


In [18]:
hydra_plus.embed_landmarks(dim, 'mpirun', nrepeat)


 Hyperbolic dimension for landmark embedding is 100.
Complete landmark time is  199.77310347557068
Best embedding error is 0.005476


(array([[-0.25201863,  0.        ,  0.        , ...,  0.03271873,
          0.37167976,  0.6916859 ],
        [-0.6415289 ,  0.        ,  0.        , ...,  0.45279471,
         -0.11406318,  0.33782216],
        [-0.53185892,  0.        ,  0.        , ...,  0.33654164,
         -0.14094454,  0.44296553],
        ...,
        [-0.19524659,  0.        ,  0.        , ...,  0.58674867,
         -0.32697834,  0.19267922],
        [-0.49629982,  0.        ,  0.        , ...,  0.2174847 ,
          0.05426408,  0.09499987],
        [-0.3020881 ,  0.        ,  0.        , ...,  0.16196927,
          0.35069668,  0.57690008]]),
 0.09109578037356222,
 array([[-0.25201863,  0.        ,  0.        , ...,  0.37167976,
          0.6916859 ,  1.96621019],
        [-0.6415289 ,  0.        ,  0.        , ..., -0.11406318,
          0.33782216,  2.15523537],
        [-0.53185892,  0.        ,  0.        , ..., -0.14094454,
          0.44296553,  1.95861758],
        ...,
        [-0.19524659,  0.       

In [19]:
hydra_plus.embed_nonlandmarks(nprocs)


Hyperbolic dimension for non-landmark embedding is 100.
Number of batches of nonlandmarks is 4
Total nonlandmark time is 0.35960 seconds
Non-landmark error:  25.559224697473148
Total nonland time:  6.234113693237305


In [20]:
'''
		validate and save validation meta data in savedir
'''
import lm_emb_hydra_functions as lm_emb_func

self = hydra_plus
nprocs3 = nprocs
MD = self.metadata
nodes = MD['nnodes']
landmarks = MD['nL']
datadir = MD['datadir']
dim = dim
# run validation test (must create /results directory)
print("begin validation tests...")
val_input = lm_emb_func.validation_preproc(nodes, landmarks, dim, nprocs3, datadir, savedir)
val_input

begin validation tests...


' -n 250 -L 101 -d 100 -p 4 -s1 /content/L-hydra/data/amazon/ -s2 /content/L-hydra/data/amazon/'

In [21]:
print("Generating random nonlandmarks...")
nonland_ids = np.array([i for i in range(nlandmarks,nNodes)])
n_test_points = nvalidation
n_nonland = len(nonland_ids.copy())
n_random = min(n_test_points, n_nonland) # choose at most 20k points
ri = rn.choice(n_nonland, n_random)
rj = rn.choice(n_nonland, n_random)
#print(ri)
#print(rj)
R = np.array([nonland_ids[ri],nonland_ids[rj]]).T
R = vstack([tuple(row) for row in R]).astype(int)
R

Generating random nonlandmarks...


array([[140, 142],
       [214, 242],
       [232, 209],
       [186, 165],
       [136, 125],
       [199, 233],
       [191, 231],
       [197, 118],
       [102, 170],
       [243, 170],
       [119, 143],
       [245, 125],
       [185, 207],
       [154, 230],
       [127, 182],
       [205, 127],
       [137, 176],
       [231, 244],
       [145, 182],
       [142, 114],
       [236, 131],
       [211, 233],
       [178, 240],
       [234, 131],
       [231, 180],
       [156, 156],
       [216, 101],
       [176, 173],
       [134, 194],
       [122, 204],
       [184, 177],
       [184, 172],
       [147, 170],
       [217, 206],
       [219, 108],
       [176, 241],
       [202, 218],
       [237, 165],
       [175, 151],
       [181, 129]])

In [22]:
Rdist = []
for edge in R:
    node1 = edge[0]
    node2 = edge[1]
    Rdist.append(D[node1,node2])
Rdist

[1.8292328371233917,
 1.4015846969131103,
 1.6534901612208581,
 1.7548159820689468,
 1.77341868020228,
 1.6300610304231733,
 1.618881758937033,
 1.6455509036588145,
 1.3993872461457488,
 1.4102936410635047,
 1.3988731152846323,
 1.5998263051622068,
 1.955655558483036,
 1.5097597920245343,
 1.527873730484221,
 1.5881467024403797,
 1.8053578116845717,
 1.7644239340642984,
 1.2678971716270333,
 1.7590500818742156,
 1.430601536621047,
 1.727158868817884,
 1.4905227931607898,
 1.409203233193053,
 1.6759004047127282,
 0.0,
 1.497192623976104,
 1.7876678519813805,
 1.5054030935649665,
 1.6636421885963764,
 1.5025695952346758,
 1.4107357033698775,
 1.4344667091981518,
 1.6363727902668197,
 1.5249874649724795,
 1.916655094770981,
 1.843816838794322,
 1.473318013883497,
 1.5739515504471862,
 1.6042401541256617]

In [23]:
savetxt(os.path.join(savedir, 'test_points.txt'),R,fmt='%i')
savetxt(os.path.join(savedir, 'test_point_distances.txt'),Rdist)

In [24]:
val_input

' -n 250 -L 101 -d 100 -p 4 -s1 /content/L-hydra/data/amazon/ -s2 /content/L-hydra/data/amazon/'

In [25]:
'''
[" -n "," -L "," -d "," -p ", " -s1 ", " -s2 "]
these flags correspond to = [nodes, landmarks, dim, nproc, dataloc, savedir]
change these if you change the values at the top of this notebook
'''

!python3 lm_emb_validation.py -n 250 -L 100 -d 2 -p 4 -s1 /content/L-hydra/data/amazon/ -s2 /content/L-hydra/data/amazon/


 ************************************************************
Embedding for n = 250 and L = 100 in Dimension 2
************************************************************
Loading data...

Validating...
  Validation error:            1.0
  Validation error w/ round.:  1.0
  Total time for validtion is  0.04361104965209961
----------------------------------------
Cleaning up data folder...
rm: cannot remove '/content/L-hydra/data/amazon/curv.txt': No such file or directory
